In [10]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.pipeline import  make_pipeline
from sksurv.metrics import concordance_index_censored,concordance_index_ipcw
from sklearn.preprocessing import StandardScaler
import joblib

In [2]:
raw_df = pd.read_csv("./data/tcga/tcga_stad.csv")

In [3]:
raw_df

,ENSG00000082929.8,ENSG00000083622.8,ENSG00000093100.13,ENSG00000099869.8,ENSG00000103472.10,ENSG00000115934.11,ENSG00000116652.6,ENSG00000117242.7,ENSG00000120664.11,ENSG00000122043.11,...,ENSG00000288648.1,ENSG00000288657.1,ENSG00000288659.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288667.1,ENSG00000288670.1,Overall Survival Status,Overall Survival (Months),recurrence
0,0.0824,0.5953,0.0739,0.0111,0.3098,0.0000,0.4138,4.3472,0.3879,0.0161,...,0.0000,0.0000,0.0000,0.5330,0.3647,2.6211,5.4329,0,27.550383,0
1,0.0000,0.0000,0.0738,0.0000,0.4453,0.0000,0.1048,2.4438,0.0106,0.0269,...,0.0000,0.0000,0.0000,0.0000,0.2522,0.0000,3.5640,1,15.024493,-1
2,0.0000,0.9866,0.0156,0.0235,0.0446,0.0000,0.3795,1.9204,0.0335,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.1479,1.6586,7.2789,1,9.238255,-1
3,0.0000,93.2180,0.0353,0.0000,0.2016,0.0000,0.0429,2.1869,0.0202,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.2053,0.0000,3.1249,0,12.657396,-1
4,0.1581,0.9178,0.1329,0.0000,0.0912,0.0000,0.1877,1.4758,0.0229,0.0000,...,0.0000,0.0000,0.0000,2.3007,0.3472,2.6401,5.6912,0,33.402374,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,0.0000,0.2273,0.0000,0.0000,0.3199,0.0000,0.1322,4.9149,0.0000,0.0216,...,0.0000,0.0000,0.0000,0.7122,0.2100,1.4011,3.7366,0,18.377881,0
243,0.2519,0.0000,0.0000,0.0000,0.5164,0.0000,0.1649,1.3197,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.2744,4.8060,3.4071,0,19.528553,0
244,0.7716,1.1944,0.1186,0.5143,0.3955,0.2896,0.4512,3.0323,0.4465,0.4210,...,0.5284,0.3346,0.5474,2.1385,0.7430,2.1034,7.0297,1,7.725943,-1
245,0.0000,0.3930,0.0455,0.0858,0.0976,0.0185,0.4641,1.3635,0.0441,0.0000,...,0.0000,0.0000,0.0000,5.7461,0.3759,1.6148,3.5433,0,13.512181,0


In [4]:
X = raw_df.iloc[:,0:-3]
y = raw_df[['Overall Survival Status', 'Overall Survival (Months)']]
y_time = raw_df['Overall Survival (Months)']  #存活時間(Overall survival time)
y_status = raw_df['Overall Survival Status'] #存活狀態(Overall survival status)

In [5]:
selected_features = ['ENSG00000162913.10', 'ENSG00000178722.13', 'ENSG00000181908.5', 'ENSG00000183242.12', 'ENSG00000185847.8', 'ENSG00000186235.11', 'ENSG00000186960.12', 'ENSG00000196166.5', 'ENSG00000197595.4', 'ENSG00000198685.4', 'ENSG00000203434.2', 'ENSG00000203987.3', 'ENSG00000204117.2', 'ENSG00000204241.8', 'ENSG00000204283.4', 'ENSG00000204677.13', 'ENSG00000204929.13', 'ENSG00000205054.8', 'ENSG00000205325.3', 'ENSG00000205396.11', 'ENSG00000205562.3', 'ENSG00000205632.4', 'ENSG00000205682.2', 'ENSG00000206187.4', 'ENSG00000213373.7', 'ENSG00000214851.5', 'ENSG00000214955.5', 'ENSG00000215162.2', 'ENSG00000215196.5', 'ENSG00000220256.4', 'ENSG00000220891.1', 'ENSG00000221845.4', 'ENSG00000222000.7', 'ENSG00000223770.5', 'ENSG00000223834.5', 'ENSG00000223930.7', 'ENSG00000224017.2', 'ENSG00000224074.3', 'ENSG00000224116.7', 'ENSG00000224153.3', 'ENSG00000224238.2', 'ENSG00000224363.2', 'ENSG00000224481.3', 'ENSG00000224819.2', 'ENSG00000224910.2', 'ENSG00000225316.3', 'ENSG00000225446.3', 'ENSG00000225465.9', 'ENSG00000225579.2', 'ENSG00000225649.6', 'ENSG00000225657.2', 'ENSG00000225683.6', 'ENSG00000225718.2', 'ENSG00000225731.1', 'ENSG00000225807.2', 'ENSG00000225914.3', 'ENSG00000226005.5', 'ENSG00000226017.2', 'ENSG00000226334.1', 'ENSG00000226397.8', 'ENSG00000226508.1', 'ENSG00000226674.11', 'ENSG00000226711.7', 'ENSG00000226758.1', 'ENSG00000226791.7', 'ENSG00000226816.3', 'ENSG00000226835.3', 'ENSG00000227082.3', 'ENSG00000227101.2', 'ENSG00000227292.1', 'ENSG00000227733.10', 'ENSG00000227769.7', 'ENSG00000227888.4', 'ENSG00000228189.5', 'ENSG00000228208.7', 'ENSG00000228274.4', 'ENSG00000228539.1', 'ENSG00000228768.3', 'ENSG00000228835.1', 'ENSG00000228971.3', 'ENSG00000228980.4', 'ENSG00000229051.2', 'ENSG00000229140.11', 'ENSG00000229262.1', 'ENSG00000229484.1', 'ENSG00000229498.1', 'ENSG00000229536.3', 'ENSG00000229611.2', 'ENSG00000229628.1', 'ENSG00000230294.4', 'ENSG00000230314.7', 'ENSG00000230768.9', 'ENSG00000230928.1', 'ENSG00000231031.6', 'ENSG00000231079.7', 'ENSG00000231170.7', 'ENSG00000231178.3', 'ENSG00000231290.6', 'ENSG00000231367.6', 'ENSG00000231682.1', 'ENSG00000231704.6', 'ENSG00000231754.2', 'ENSG00000232046.7', 'ENSG00000232079.7', 'ENSG00000232117.2', 'ENSG00000232294.6', 'ENSG00000233061.1', 'ENSG00000233073.2', 'ENSG00000233086.8', 'ENSG00000233288.2', 'ENSG00000233508.3', 'ENSG00000233723.9', 'ENSG00000233766.7', 'ENSG00000233977.1', 'ENSG00000234089.2', 'ENSG00000234215.3', 'ENSG00000234255.9', 'ENSG00000234323.8', 'ENSG00000234418.2', 'ENSG00000234423.2', 'ENSG00000234698.1', 'ENSG00000234699.1', 'ENSG00000234721.3', 'ENSG00000234722.5', 'ENSG00000234948.2', 'ENSG00000235009.1', 'ENSG00000235023.1', 'ENSG00000235054.4', 'ENSG00000235120.1', 'ENSG00000235158.1', 'ENSG00000235295.1', 'ENSG00000235532.1', 'ENSG00000235535.8', 'ENSG00000235590.7', 'ENSG00000235660.2', 'ENSG00000235770.6', 'ENSG00000235813.1', 'ENSG00000235824.5', 'ENSG00000235881.2', 'ENSG00000235904.3', 'ENSG00000235979.9', 'ENSG00000236054.1', 'ENSG00000236107.10', 'ENSG00000236197.4', 'ENSG00000236230.2', 'ENSG00000236333.4', 'ENSG00000236451.3', 'ENSG00000236467.8', 'ENSG00000236839.1', 'ENSG00000236882.8', 'ENSG00000236886.2', 'ENSG00000236990.1', 'ENSG00000237063.1', 'ENSG00000237286.1', 'ENSG00000237390.1', 'ENSG00000237480.3', 'ENSG00000237525.7', 'ENSG00000237574.1', 'ENSG00000237595.5', 'ENSG00000237667.7', 'ENSG00000237975.7', 'ENSG00000238217.6', 'ENSG00000238290.1', 'ENSG00000238755.3', 'ENSG00000239828.7', 'ENSG00000240057.6', 'ENSG00000241059.2', 'ENSG00000241098.1', 'ENSG00000241131.2', 'ENSG00000241383.2', 'ENSG00000241469.9', 'ENSG00000242512.9', 'ENSG00000242536.2', 'ENSG00000242540.3', 'ENSG00000242578.1', 'ENSG00000242880.2', 'ENSG00000243150.6', 'ENSG00000243620.2', 'ENSG00000243836.5', 'ENSG00000244578.1', 'ENSG00000244968.7', 'ENSG00000245105.4', 'ENSG00000245479.3', 'ENSG00000245526.11', 'ENSG00000245685.7', 'ENSG00000246350.1', 'ENSG00000246695.8', 'ENSG00000246731.3', 'ENSG00000247121.7', 'ENSG00000247809.9', 'ENSG00000248174.6', 'ENSG00000248360.7', 'ENSG00000248455.6', 'ENSG00000248456.1', 'ENSG00000248505.2', 'ENSG00000248550.4', 'ENSG00000248572.6', 'ENSG00000249125.2', 'ENSG00000249258.2', 'ENSG00000249406.3', 'ENSG00000249738.10', 'ENSG00000249917.3', 'ENSG00000249926.2', 'ENSG00000250258.1', 'ENSG00000250260.1', 'ENSG00000250377.2', 'ENSG00000250486.5', 'ENSG00000250961.2', 'ENSG00000250992.2', 'ENSG00000251072.2', 'ENSG00000251139.2', 'ENSG00000251216.2', 'ENSG00000251372.6', 'ENSG00000251408.1', 'ENSG00000251448.2', 'ENSG00000251538.7', 'ENSG00000251600.8', 'ENSG00000251629.7', 'ENSG00000251688.2', 'ENSG00000253235.2', 'ENSG00000253238.3', 'ENSG00000253252.2', 'ENSG00000253452.6', 'ENSG00000253496.3', 'ENSG00000253604.1', 'ENSG00000253712.1', 'ENSG00000253959.1', 'ENSG00000254006.5', 'ENSG00000254040.2', 'ENSG00000254041.1', 'ENSG00000254092.2', 'ENSG00000254162.1', 'ENSG00000254226.6', 'ENSG00000254251.1', 'ENSG00000254344.3', 'ENSG00000254416.6', 'ENSG00000254427.2', 'ENSG00000254561.4', 'ENSG00000254602.2', 'ENSG00000254775.2', 'ENSG00000254839.1', 'ENSG00000254842.6', 'ENSG00000255130.1', 'ENSG00000255226.2', 'ENSG00000255652.4', 'ENSG00000255727.2', 'ENSG00000255829.1', 'ENSG00000255850.2', 'ENSG00000256120.2', 'ENSG00000256193.6', 'ENSG00000256343.8', 'ENSG00000256422.6', 'ENSG00000256654.4', 'ENSG00000257060.7', 'ENSG00000257083.7', 'ENSG00000257359.1', 'ENSG00000257398.8', 'ENSG00000257530.1', 'ENSG00000257599.3', 'ENSG00000258034.1', 'ENSG00000258390.2', 'ENSG00000258422.5', 'ENSG00000258473.2', 'ENSG00000258479.6', 'ENSG00000258583.7', 'ENSG00000258603.4', 'ENSG00000258654.1', 'ENSG00000258702.3', 'ENSG00000258776.1', 'ENSG00000258888.1', 'ENSG00000259070.7', 'ENSG00000259103.2', 'ENSG00000259129.6', 'ENSG00000259130.2', 'ENSG00000259235.2', 'ENSG00000259280.1', 'ENSG00000259363.6', 'ENSG00000259476.1', 'ENSG00000259478.2', 'ENSG00000259503.1', 'ENSG00000259639.6', 'ENSG00000259727.1', 'ENSG00000259763.2', 'ENSG00000259887.1', 'ENSG00000259905.7', 'ENSG00000260126.2', 'ENSG00000260198.1', 'ENSG00000260206.1', 'ENSG00000260271.3', 'ENSG00000260340.1', 'ENSG00000260366.1', 'ENSG00000260477.8', 'ENSG00000260517.3', 'ENSG00000260569.2', 'ENSG00000260599.1', 'ENSG00000260679.1', 'ENSG00000260737.6', 'ENSG00000260741.1', 'ENSG00000260876.6', 'ENSG00000260930.2', 'ENSG00000260989.1', 'ENSG00000261033.1', 'ENSG00000261058.1', 'ENSG00000261090.2', 'ENSG00000261175.6', 'ENSG00000261348.2', 'ENSG00000261402.1', 'ENSG00000261445.1', 'ENSG00000261519.3', 'ENSG00000261615.7', 'ENSG00000261623.1', 'ENSG00000261653.1', 'ENSG00000261671.1', 'ENSG00000261730.1', 'ENSG00000261821.2', 'ENSG00000262267.6', 'ENSG00000262619.2', 'ENSG00000262801.6', 'ENSG00000263293.2', 'ENSG00000263547.2', 'ENSG00000263923.1', 'ENSG00000264589.4', 'ENSG00000264739.1', 'ENSG00000264954.2', 'ENSG00000265313.7', 'ENSG00000265519.1', 'ENSG00000265758.1', 'ENSG00000265801.1', 'ENSG00000266279.1', 'ENSG00000266368.1', 'ENSG00000266369.1', 'ENSG00000266973.1', 'ENSG00000267207.1', 'ENSG00000267272.5', 'ENSG00000267339.7', 'ENSG00000267391.5', 'ENSG00000267409.1', 'ENSG00000267429.1', 'ENSG00000267489.5', 'ENSG00000267549.6', 'ENSG00000267750.6', 'ENSG00000267774.2', 'ENSG00000267890.1', 'ENSG00000268006.1', 'ENSG00000268333.1', 'ENSG00000268355.1', 'ENSG00000268894.7', 'ENSG00000269194.1', 'ENSG00000269506.2', 'ENSG00000269543.5', 'ENSG00000269842.5', 'ENSG00000269889.1', 'ENSG00000269898.1', 'ENSG00000270562.1', 'ENSG00000271171.3', 'ENSG00000271538.6', 'ENSG00000271584.2', 'ENSG00000271860.7', 'ENSG00000272482.1', 'ENSG00000272691.1', 'ENSG00000272797.1', 'ENSG00000272814.1', 'ENSG00000272865.2', 'ENSG00000273372.1', 'ENSG00000274253.5', 'ENSG00000274827.4', 'ENSG00000275016.5', 'ENSG00000275427.1', 'ENSG00000275805.1', 'ENSG00000275894.1', 'ENSG00000276397.2', 'ENSG00000277270.2', 'ENSG00000277301.1', 'ENSG00000277763.1', 'ENSG00000277829.1', 'ENSG00000279047.4', 'ENSG00000279217.1', 'ENSG00000279261.2', 'ENSG00000280011.1', 'ENSG00000280025.1', 'ENSG00000280711.1', 'ENSG00000282602.1', 'ENSG00000283213.1', 'ENSG00000283380.1', 'ENSG00000283462.2', 'ENSG00000283503.2', 'ENSG00000283538.3', 'ENSG00000284601.1', 'ENSG00000284612.1', 'ENSG00000284640.1', 'ENSG00000284668.2', 'ENSG00000284675.1', 'ENSG00000284699.1', 'ENSG00000284703.1', 'ENSG00000284716.1', 'ENSG00000284848.2', 'ENSG00000285090.1', 'ENSG00000285169.2', 'ENSG00000285294.1', 'ENSG00000285336.1', 'ENSG00000285347.1', 'ENSG00000285454.1', 'ENSG00000285486.1', 'ENSG00000285520.1', 'ENSG00000285523.1', 'ENSG00000285638.1', 'ENSG00000285801.1', 'ENSG00000285829.1', 'ENSG00000285838.2', 'ENSG00000285876.1', 'ENSG00000285894.2', 'ENSG00000285908.1', 'ENSG00000285909.1', 'ENSG00000285925.1', 'ENSG00000285929.1', 'ENSG00000285936.1', 'ENSG00000285980.2', 'ENSG00000286003.1', 'ENSG00000286044.1', 'ENSG00000286125.1', 'ENSG00000286195.1', 'ENSG00000286198.1', 'ENSG00000286212.2', 'ENSG00000286213.1', 'ENSG00000286248.1', 'ENSG00000286292.1', 'ENSG00000286293.1', 'ENSG00000286313.1', 'ENSG00000286376.1', 'ENSG00000286391.1', 'ENSG00000286431.1', 'ENSG00000286450.1', 'ENSG00000286460.1', 'ENSG00000286461.1', 'ENSG00000286517.1', 'ENSG00000286521.1', 'ENSG00000286565.1', 'ENSG00000286596.1', 'ENSG00000286641.1', 'ENSG00000286648.1', 'ENSG00000286654.1', 'ENSG00000286703.1', 'ENSG00000286710.1', 'ENSG00000286717.1', 'ENSG00000286734.1', 'ENSG00000286788.1', 'ENSG00000286810.1', 'ENSG00000286818.1', 'ENSG00000286819.1', 'ENSG00000286915.1', 'ENSG00000287062.1', 'ENSG00000287076.1', 'ENSG00000287079.1', 'ENSG00000287089.1', 'ENSG00000287161.1', 'ENSG00000287193.1', 'ENSG00000287232.1', 'ENSG00000287278.1', 'ENSG00000287279.1', 'ENSG00000287313.1', 'ENSG00000287354.1', 'ENSG00000287378.1', 'ENSG00000287393.1', 'ENSG00000287447.1', 'ENSG00000287498.1', 'ENSG00000287525.1', 'ENSG00000287550.1', 'ENSG00000287706.1', 'ENSG00000287795.1', 'ENSG00000287797.1', 'ENSG00000287798.1', 'ENSG00000287804.1', 'ENSG00000287826.1', 'ENSG00000287836.1', 'ENSG00000287896.1', 'ENSG00000287982.1', 'ENSG00000287996.1', 'ENSG00000288022.1', 'ENSG00000288045.1', 'ENSG00000288079.1', 'ENSG00000288106.1', 'ENSG00000288253.1', 'ENSG00000288560.1', 'ENSG00000288575.1']

In [6]:
len(selected_features)

484

In [12]:
with open('./hw2/tcga-survival-features.csv', 'w') as f:
    for idx, feature in enumerate(selected_features):
        if idx < len(selected_features)-1: f.write(f"{feature},")
        else: f.write(f"{feature}")

In [7]:
X_train = X[selected_features]
y_train = y.to_records(index=False).astype([('Status', 'bool'), ('Survival', 'float64')])

In [8]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
cox = CoxPHSurvivalAnalysis(alpha=0.001)
cox.fit(X_train,y_train)
pred = cox.predict(X_train)

c:\Users\Azetry\anaconda3\envs\ml-exp\lib\site-packages\sksurv\linear_model\coxph.py:177: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
c:\Users\Azetry\anaconda3\envs\ml-exp\lib\site-packages\sksurv\linear_model\coxph.py:174: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])


In [9]:
print(concordance_index_censored(y_status>0,y_time,pred))

(1.0, 15083, 0, 0, 12)


In [11]:
joblib.dump(cox, "./hw2/tcga-survival-cox.model")

['./hw2/tcga-survival-cox.model']